# ДЗ 3 
## Ранжирование и BM25

## Функция ранжирования BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Query$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $Doc$ запросу $Query$:

$$ BM25(Query, Doc) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,Doc)*(k+1)}{TF(q_i,Doc)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где    
$$$$
$\text{IDF}(q_i)$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в корпусе   
$n(q_i)$ — количество документов, содержащих слово $q_i$

>$TF(q_i,Doc)$ - частота слова $q_i$ в документе $Doc$    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75  
$l(d)$ - длина документа (количество слов в нём)   
$avgdl$ — средняя длина документа в корпусе    

### __Задача__:

Реализуйте поиск, где
- в качестве векторизации документов корпуса - слагаемые **BM25**
- формат хранения индекса - **матрица Document-Term**
- метрика близости пар (запрос, документ) - **BM25**

В реализации должно быть все то же, что во втором дз:
- функция индексации корпуса, на выходе которой посчитанная матрица Document-Term
- функция индексации запроса, на выходе которой посчитанный вектор запроса
- функция с реализацией подсчета близости запроса и документов корпуса, на выходе которой вектор, i-й элемент которого обозначает близость запроса с i-м документом корпуса. Сделать **обязательно векторно**.
- главная функция, объединяющая все это вместе; на входе - запрос, на выходе - отсортированные по убыванию имена документов коллекции

Обратите внимание:
- сортировку надо сделать **<font color='green'>обязательно векторно</font>** через маску; при не соблюдении минус два балла
- подсчет индекса надо сделать **<font color='green'>обязательно с использованием спарсованных матриц</font>**, то есть ни в какой момент времени векторизованный корпус не переводится в ndarray; при не соблюдении минус два балла
- надо сделать **<font color='green'>обязательно с учетом всех комментариев за свои предыдущие дз</font>**; при не соблюдении минус балл за каждый пункт
- использование хардкод локального пути => 0 баллов


В качестве корпуса возьмите корпус вопросов и ответов с Ответы Мейл) 👍😃 
[Ссылка для скачивания](https://www.kaggle.com/bobazooba/thousands-of-questions-about-love)

Описание структуры данных можно посмотреть по ссылке. В качестве документов корпуса берем значения из ключа *answers*, но не все, а один, у которого максимальный *value*. При этом ограничиваем количество документов до 50000. Пример ниже.


**На что направлена эта задача:** 
Реализация поисковика с использованием BM25.



In [31]:
# # датасет для индексирования

import json

with open('questions_about_love.jsonl', 'r') as f:
    corpus = list(f)[:50000]

# пример элемента оригинального датасета 
json.loads(corpus[22])

{'question': 'Кто что должен сделать!? Для завоевания доверия женщины у мужчины и мужчины у женщины, для прочных отношений!',
 'comment': '',
 'sub_category': 'relations',
 'author': 'Diesel',
 'author_rating': {'category': 'Мыслитель', 'value': '5175'},
 'answers': [{'text': 'это подозрительно когда настойчиво навязывают чувство доверия',
   'author_rating': {'category': 'Знаток', 'value': '312'}},
  {'text': 'Пересказ информации вайшнавов. Доверие складывается из 2 штук. 1. Доброта - пилот добрый, но не умеет водить самолет - лететь страшно, доверия нет. 2. Профессионализм - зирург отличный, но садист, отрежет лишнее - нет доверия.Итак, учитывайте потребности человека, повышайте айкью, чтоб по внешнему виду определять че человеку надо, не плюйте на его состояние (не просите больного гриппом идти для вас в аптеку за презиками), покажите, что вы никогда не будете его насиловать - в случае если вас что-то не устроит просто уйдете. Говорите правду. Желательно не косячить - такую правду г

In [32]:
# подсказки про векторную сортировку

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


corpus = [
    'мыла',
    'раму',
    'киса',
    'мама мыла раму'
]
corpus_doc_names = np.array(['name_1', 'name_2', 'name_3', 'name_4'])
query = 'мама мыла раму'


vectorizer = TfidfVectorizer()
corpus_matrix = vectorizer.fit_transform(corpus)
query_vec = vectorizer.transform([query]).toarray()

# считаем косинусную близость
scores = cosine_similarity(corpus_matrix, query_vec)

# сортируем индексы скоров в обратном порядке (по убыванию)
sorted_scores_indx = np.argsort(scores, axis=0)[::-1]

# сортируем имена файлов в соответствии со скорами
corpus_doc_names[sorted_scores_indx.ravel()]

array(['name_4', 'name_2', 'name_1', 'name_3'], dtype='<U6')

In [26]:
# подсказки про спарс-матрицу

from scipy import sparse


# итерация по ненулевым элементам спарс-матрицы
# for i, j in zip(*sparce_matrix.nonzero()): 
#     ...
    
# создать спарс-матрицу из данных, где
# values - лист из n значений, которые мы хотим положить в матрицу 
# rows - лист из n значений, где i-тое значение это индекс строки i-го элемента из values
# cols - лист из n значений, где i-тое значение это индекс колонки i-го элемента из values

values = [99, 22, 77]
rows = [0, 2, 3]
cols = [0, 2, 4]

sparce_matrix = sparse.csr_matrix((values, (rows, cols)))
sparce_matrix

<4x5 sparse matrix of type '<class 'numpy.longlong'>'
	with 3 stored elements in Compressed Sparse Row format>

In [27]:
# и посмотрим, что получилось
sparce_matrix.toarray()

array([[99,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0, 22,  0,  0],
       [ 0,  0,  0,  0, 77]], dtype=int64)

In [29]:
for i, j in zip(*sparce_matrix.nonzero()): 
    print(i, j)

0 0
2 2
3 4
